In [40]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

In [41]:
abb_df = pd.read_csv('/Users/nli/dev/airflow_home/data/abb_nyc_housing_2019_data.csv')

In [42]:
abb_df.shape

(48895, 17)

In [43]:
abb_df.head()

,Unnamed: 0,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [44]:
abb_df.describe()

,Unnamed: 0,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,48895.000000,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,24447.000000,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,14114.915043,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,0.000000,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,12223.500000,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,24447.000000,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,36670.500000,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,48894.000000,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [45]:
#remove those price euqla $0
abb_df = abb_df[abb_df['price']>0]

In [46]:
#zremove minimum_night greater than 365 days
abb_df = abb_df[abb_df['minimum_nights']<=365]

In [47]:
abb_df.dtypes

Unnamed: 0                          int64
id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [48]:
abb_df.isnull().sum()

Unnamed: 0                            0
id                                    0
name                                 15
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10043
reviews_per_month                 10043
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [49]:
# the revires_month data seems good for future analysis, 
# we replace the missing value as 0, we will drop the last_review column
abb_df.loc[abb_df['reviews_per_month'].isnull(),'reviews_per_month'] = 0

In [50]:
abb_df.isnull().sum()

Unnamed: 0                            0
id                                    0
name                                 15
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10043
reviews_per_month                     0
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [51]:
abb_df.nunique()

Unnamed: 0                        48870
id                                48870
name                              47883
host_id                           37442
host_name                         11448
neighbourhood_group                   5
neighbourhood                       221
latitude                          19045
longitude                         14715
room_type                             3
price                               673
minimum_nights                      101
number_of_reviews                   394
last_review                        1764
reviews_per_month                   938
calculated_host_listings_count       47
availability_365                    366
dtype: int64

In [52]:
#From the table, we have missing data from name and host_name. We just drop the skipped this two columns.
#All id seems unqiue, but not host_id. So, we just renomve he id column.
abb_df2 = abb_df.drop(columns=['id', 'name', 'host_name', 'last_review'])

In [53]:
abb_df2.isnull().sum()

Unnamed: 0                        0
host_id                           0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [54]:
abb_df2.rename(columns={'neighbourhood_group':'borough'}, inplace = True)

In [55]:
abb_df.head()

,Unnamed: 0,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,0.00,1,365
3,3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [56]:
#All data are filled and cleaned.
abb_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48870 entries, 0 to 48894
Data columns (total 13 columns):
Unnamed: 0                        48870 non-null int64
host_id                           48870 non-null int64
borough                           48870 non-null object
neighbourhood                     48870 non-null object
latitude                          48870 non-null float64
longitude                         48870 non-null float64
room_type                         48870 non-null object
price                             48870 non-null int64
minimum_nights                    48870 non-null int64
number_of_reviews                 48870 non-null int64
reviews_per_month                 48870 non-null float64
calculated_host_listings_count    48870 non-null int64
availability_365                  48870 non-null int64
dtypes: float64(3), int64(7), object(3)
memory usage: 5.2+ MB


In [57]:
#Now we have follow rows and columns
abb_df2.shape

(48870, 13)

In [58]:
abb_df2.tail()

,Unnamed: 0,host_id,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
48890,48890,8232441,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,0.0,2,9
48891,48891,6570630,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,0.0,2,36
48892,48892,23492952,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,0.0,1,27
48893,48893,30985759,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,0.0,6,2
48894,48894,68119814,Manhattan,Hell's Kitchen,40.76404,-73.98933,Private room,90,7,0,0.0,1,23


In [59]:
# Next, we will look at the price by the different boroughs
abb_df2.borough.unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Bronx'],
      dtype=object)

In [60]:
set(abb_df2['borough'])

{'Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'}

In [61]:
abb_df2.groupby('borough')['price'].describe()

,count,mean,std,min,25%,50%,75%,max
borough,,,,,,,,
Bronx,1090.0,87.577064,106.725371,10.0,45.0,65.0,99.0,2500.0
Brooklyn,20089.0,124.452238,186.922112,10.0,60.0,90.0,150.0,10000.0
Manhattan,21654.0,196.888011,291.421157,10.0,95.0,150.0,220.0,10000.0
Queens,5664.0,99.493997,167.125802,10.0,50.0,75.0,110.0,10000.0
Staten Island,373.0,114.812332,277.620403,13.0,50.0,75.0,110.0,5000.0


In [62]:
#The total neighthourhood number:
len(set(abb_df['neighbourhood']))

221

In [63]:
#Next, we look at the room type
abb_df_price_pivot=abb_df2.pivot_table(index=['borough'], values='price', columns='room_type', aggfunc='mean')

In [64]:
abb_df_price_pivot

room_type,Entire home/apt,Private room,Shared room
borough,,,
Bronx,127.506596,66.890937,59.800000
Brooklyn,178.356844,76.553547,50.773723
Manhattan,249.276359,116.776622,88.933194
Queens,147.031996,71.762456,69.020202
Staten Island,173.846591,62.292553,57.444444


In [65]:
#Let's look deep in the neightbooh 
#abb_df2.pivot_table(index=['borough','neighbourhood'], values='price', columns='room_type', aggfunc='mean')

In [66]:
abb_df_price_pivot.to_csv('/Users/nli/dev/airflow_home/data/abb_df_borough_pivot_table.csv')

In [67]:
#reset the index as host_id
abb_df2 = abb_df2.set_index('host_id')

In [68]:
abb_df2.head()

,Unnamed: 0,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
host_id,,,,,,,,,,,,
2787,0,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
2845,1,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
4632,2,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,0.00,1,365
4869,3,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
7192,4,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0


In [71]:
abb_df2 = abb_df2.loc[:, ~ abb_df2.columns.str.contains('^Unnamed')]
abb_df2.head()

,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
host_id,,,,,,,,,,,
2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,0.00,1,365
4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0


In [72]:
abb_df2.to_csv('/Users/nli/dev/airflow_home/data/abb_df2.csv')